# Calling Library

In [1]:
import numpy as np
import pandas as pd
import pyodbc 
import getpass
import pandas.io.sql

# For Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib


# for saving the pipeline
import joblib

# from Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline 


# from feature-engine
from feature_engine.imputation import (
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer,
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder,
)

from feature_engine.transformation import (
    LogTransformer,
    YeoJohnsonTransformer,
)

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper


# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Pandas Settings
# pd.options.display.float_format = '{:.1f}'.format
pd.set_option('display.max_rows', 40000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

# Miscellaneous imports
from datetime import datetime
from scipy.stats import norm
import re



# Database Connection

In [2]:
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'IN3031255W1\SQLEXPRESS' 
database = 'Advance_Analytics' 
username = input(" Enter username: ")
password = getpass.getpass(" Enter Password: ")

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                       SERVER='+server+';\
                       DATABASE='+database+';\
                       Trusted_Connection=yes;')


cursor = cnxn.cursor()

# cnxn.close()

 Enter username: MEA\CT135EY
 Enter Password: ········


## Reading Tables within a Database to Dataframes

In [3]:
read='select * from ConsumerElectronics'
# directly read into dataframe
df_Consumer_electronics= pandas.io.sql.read_sql(read, cnxn)
df_Consumer_electronics.head()
# cnxn.close()


,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,cust_id,pincode,product_analytic_super_category,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
0,ACCCX3S58G7B5F6P,2015-10-17 15:11:54,2015,10,3.419301e+15,3.419301e+15,6400,1,\N,\N,COD,5,-1.01299130778588E+018,-7.79175582905735E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
1,ACCCX3S58G7B5F6P,2015-10-19 10:07:22,2015,10,1.420831e+15,1.420831e+15,6900,1,\N,\N,COD,7,-8.99032457905512E+018,7.33541149097431E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
2,ACCCX3S5AHMF55FV,2015-10-20 15:45:56,2015,10,2.421913e+15,2.421913e+15,1990,1,\N,\N,COD,10,-1.0404429420466E+018,-7.47768776228657E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
3,ACCCX3S5AHMF55FV,2015-10-14 12:05:15,2015,10,4.416592e+15,4.416592e+15,1690,1,\N,\N,Prepaid,4,-7.60496084352714E+018,-5.83593163877661E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
4,ACCCX3S5AHMF55FV,2015-10-17 21:25:03,2015,10,4.419525e+15,4.419525e+15,1618,1,\N,\N,Prepaid,6,2.8945572083453E+018,5.34735360997242E+017,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3


In [278]:
df_Consumer_electronics1=df_Consumer_electronics

In [279]:
# reduce un wanted columns

df_Consumer_electronics1=df_Consumer_electronics1.drop(['fsn_id','order_id','order_item_id',
                                                      'cust_id','pincode','product_analytic_super_category',
                                                        'product_procurement_sla'],axis=1)
df_Consumer_electronics1


,order_date,Year,Month,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp
0,2015-10-17 15:11:54,2015,10,6400,1,\N,\N,COD,5,CameraAccessory,CameraAccessory,CameraTripod,7190
1,2015-10-19 10:07:22,2015,10,6900,1,\N,\N,COD,7,CameraAccessory,CameraAccessory,CameraTripod,7190
2,2015-10-20 15:45:56,2015,10,1990,1,\N,\N,COD,10,CameraAccessory,CameraAccessory,CameraTripod,2099
3,2015-10-14 12:05:15,2015,10,1690,1,\N,\N,Prepaid,4,CameraAccessory,CameraAccessory,CameraTripod,2099
4,2015-10-17 21:25:03,2015,10,1618,1,\N,\N,Prepaid,6,CameraAccessory,CameraAccessory,CameraTripod,2099
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648819,2015-09-27 19:20:31,2015,9,2200,1,\N,\N,COD,7,EntertainmentSmall,TVVideoSmall,VideoPlayer,2499
1648820,2015-09-28 01:32:58,2015,9,2098,1,\N,\N,COD,10,EntertainmentSmall,TVVideoSmall,VideoPlayer,2499
1648821,2015-09-29 07:07:57,2015,9,2098,1,\N,\N,COD,5,EntertainmentSmall,TVVideoSmall,VideoPlayer,2499
1648822,2015-09-29 18:15:45,2015,9,2200,1,\N,\N,COD,3,EntertainmentSmall,TVVideoSmall,VideoPlayer,2499


In [280]:
# reduce un wanted rows
#by selecting Camera & CameraAccessory from product_analytic_category column
# select data for 3 months

df_Consumer_electronics1=df_Consumer_electronics1.loc[(df_Consumer_electronics1['product_analytic_category']=='CameraAccessory') | (df_Consumer_electronics1['product_analytic_category']=='Camera')]


df_Consumer_electronics1= df_Consumer_electronics1.loc[(df_Consumer_electronics1['order_date'] >= '2015-06-26') & (df_Consumer_electronics1['order_date'] <'2015-12-26')]

df_Consumer_electronics1

,order_date,Year,Month,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp
0,2015-10-17 15:11:54,2015,10,6400,1,\N,\N,COD,5,CameraAccessory,CameraAccessory,CameraTripod,7190
1,2015-10-19 10:07:22,2015,10,6900,1,\N,\N,COD,7,CameraAccessory,CameraAccessory,CameraTripod,7190
2,2015-10-20 15:45:56,2015,10,1990,1,\N,\N,COD,10,CameraAccessory,CameraAccessory,CameraTripod,2099
3,2015-10-14 12:05:15,2015,10,1690,1,\N,\N,Prepaid,4,CameraAccessory,CameraAccessory,CameraTripod,2099
4,2015-10-17 21:25:03,2015,10,1618,1,\N,\N,Prepaid,6,CameraAccessory,CameraAccessory,CameraTripod,2099
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645038,2015-09-24 10:06:09,2015,9,2990,1,\N,\N,COD,11,CameraAccessory,CameraAccessory,Telescope,8500
1645039,2015-09-02 19:32:02,2015,9,2990,1,\N,\N,COD,8,CameraAccessory,CameraAccessory,Telescope,8500
1645040,2015-09-14 16:27:04,2015,9,2990,1,\N,\N,COD,6,CameraAccessory,CameraAccessory,Telescope,8500
1645041,2015-09-24 12:44:24,2015,9,2990,1,\N,\N,COD,6,CameraAccessory,CameraAccessory,Telescope,8500


In [ ]:
read='select * from Media_Investment'
# directly read into dataframe
df_media_investment= pandas.io.sql.read_sql(read, cnxn)
df_media_investment.head()
# cnxn.close()


In [ ]:
read='select * from Monthly_NPS_Score'
# directly read into dataframe
df_monthly_nps_score= pandas.io.sql.read_sql(read, cnxn)
df_monthly_nps_score.head()
# cnxn.close()


In [ ]:
read='select * from Product_List'
# directly read into dataframe
df_product_list= pandas.io.sql.read_sql(read, cnxn)
df_product_list.head()
# cnxn.close()


In [ ]:
read='select * from Special_Sale_Calendar'
# directly read into dataframe
df_special_sale_calendar= pandas.io.sql.read_sql(read, cnxn)
df_special_sale_calendar.head()
# cnxn.close()


# Data Preprocessing

### Functions for Ingestion in Pipeline

In [349]:
data=df_Consumer_electronics1
data.head()

,order_date,Year,Month,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp
0,2015-10-17 15:11:54,2015,10,6400.0,1,\N,\N,COD,5,CameraAccessory,CameraAccessory,CameraTripod,7190
1,2015-10-19 10:07:22,2015,10,6900.0,1,\N,\N,COD,7,CameraAccessory,CameraAccessory,CameraTripod,7190
2,2015-10-20 15:45:56,2015,10,1990.0,1,\N,\N,COD,10,CameraAccessory,CameraAccessory,CameraTripod,2099
3,2015-10-14 12:05:15,2015,10,1690.0,1,\N,\N,Prepaid,4,CameraAccessory,CameraAccessory,CameraTripod,2099
4,2015-10-17 21:25:03,2015,10,1618.0,1,\N,\N,Prepaid,6,CameraAccessory,CameraAccessory,CameraTripod,2099


In [ ]:
# removing rows with 0, nan and '' 
def replace_missing_val(data):
    data.replace(' ', np.nan, inplace = True)
    data = data[~pd.isnull(data['gmv'])]    
    return data

In [ ]:
missing_value=preprocessing.FunctionTransformer(replace_missing_val)

In [ ]:
# converting columns to proper datatypes
def datatype_conversion(data):
#         df = pd.DataFrame(data, columns=gmv)
        #int to string
#         data[['order_id','order_item_bid']] = data[['order_id','order_item_id']].astype(object) not needed
        data[['Year','Month']] = data[['Year','Month']].astype(str)
        data['gmv'] = pd.to_numeric(data['gmv'], errors='coerce')
        data["order_date"]=data["order_date"].astype("object")               
        return data

In [ ]:
datatype_transformer=preprocessing.FunctionTransformer(datatype_conversion)

In [ ]:
# for product mrp fixing invalid values
def invalid_computation(data):
    # update column based on another column
    data.product_mrp = np.where(data['product_mrp'] < data['gmv'] / data['units'], \
                                       data['gmv'] / data['units'], data['product_mrp'])
#     data = data.loc[(data['deliverybdays'] >= 0) & (data['deliverycdays'] >= 0)]
#     data.reset_index(drop=True, inplace=True)
#     data = data.loc[(data['product_procurement_sla'] >= 0)]
#     data.reset_index(drop=True, inplace=True)
#     data = data.loc[(data['product_procurement_sla'] < 1000)]
    return data 

In [ ]:
invalid_compute=preprocessing.FunctionTransformer(invalid_computation)

###  Building Pipline

In [376]:
# set up the pipeline
new_pipe = Pipeline([
    # datetime format is not able to process in pipeline

    # ===== IMPUTATION =====
    #simple imputation by filling with 0
#     ( "missing_imputation", SimpleImputer
#     (missing_values=np.nan, add_indicator=True, strategy="constant", fill_value=0)),
    ("missing_value_treatment",missing_value)
    
    
    # ===== Data Type Conversion =====
   ("datatype_conversion",datatype_transformer)
    
    
#     # ===== Fixing Invalid Values =====
   ("fix_invalid_vals",invalid_compute)   
    
])

In [377]:
new_pipe.fit(data)

Pipeline(steps=[('fix_invalid_vals',
                 FunctionTransformer(func=<function invalid_computation at 0x000002C8FECBCD30>))])

In [378]:
x=new_pipe.transform(data)
x

,order_date,Year,Month,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp
0,2015-10-17 15:11:54,2015,10,6400.0,1,\N,\N,COD,5,CameraAccessory,CameraAccessory,CameraTripod,7190.0
1,2015-10-19 10:07:22,2015,10,6900.0,1,\N,\N,COD,7,CameraAccessory,CameraAccessory,CameraTripod,7190.0
2,2015-10-20 15:45:56,2015,10,1990.0,1,\N,\N,COD,10,CameraAccessory,CameraAccessory,CameraTripod,2099.0
3,2015-10-14 12:05:15,2015,10,1690.0,1,\N,\N,Prepaid,4,CameraAccessory,CameraAccessory,CameraTripod,2099.0
4,2015-10-17 21:25:03,2015,10,1618.0,1,\N,\N,Prepaid,6,CameraAccessory,CameraAccessory,CameraTripod,2099.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645038,2015-09-24 10:06:09,2015,9,2990.0,1,\N,\N,COD,11,CameraAccessory,CameraAccessory,Telescope,8500.0
1645039,2015-09-02 19:32:02,2015,9,2990.0,1,\N,\N,COD,8,CameraAccessory,CameraAccessory,Telescope,8500.0
1645040,2015-09-14 16:27:04,2015,9,2990.0,1,\N,\N,COD,6,CameraAccessory,CameraAccessory,Telescope,8500.0
1645041,2015-09-24 12:44:24,2015,9,2990.0,1,\N,\N,COD,6,CameraAccessory,CameraAccessory,Telescope,8500.0


# Feauture Engineering

## Functions for ingestion in Pipeline

In [ ]:
def list_index(data):
    loc_index = data.columns.get_loc('gmv') + 1
    data.insert(loc=loc_index,column='list_price',value = data['gmv'] / data['units'])
    return data

In [ ]:
list_price_fe=preprocessing.FunctionTransformer(list_index)

In [ ]:
def pay_day_flag(data):
    data['payday_flag'] = data['order_date'].apply(lambda x:1 if x.strftime('%d') in ('14','15','16','30','31','1','2') else 0)
    return data

In [ ]:
pd_flag_fe=preprocessing.FunctionTransformer(pay_day_flag)

In [ ]:
def holiday_fe(data):
    data['occassion_flag'] = data['order_date'].apply(lambda x:holidayflg(x))
    return data

In [ ]:

        def holidayflg(ord_date):
            if ord_date.strftime('%Y') == 2015:
                if ord_date.strftime('%m') == '07' and ord_date.strftime('%d') == '01':
                    return 1
                elif ord_date.strftime('%m') == '08' and ord_date.strftime('%d') == '03':
                    return 1
                elif ord_date.strftime('%m') == '09' and ord_date.strftime('%d') == '07':
                    return 1
                elif ord_date.strftime('%m') == '10' and ord_date.strftime('%d') == '12':
                    return 1
                elif ord_date.strftime('%m') == '10' and ord_date.strftime('%d') == '31':
                    return 1
                elif ord_date.strftime('%m') == '11' and ord_date.strftime('%d') == '11':
                    return 1
                elif ord_date.strftime('%m') == '12' and ord_date.strftime('%d') == '25':
                    return 1
                elif ord_date.strftime('%m') == '12' and ord_date.strftime('%d') == '26':
                    return 1
                else:
                    return 0
            else:
                if ord_date.strftime('%m') == '01' and ord_date.strftime('%d') == '01':
                    return 1
                elif ord_date.strftime('%m') == '02' and ord_date.strftime('%d') == '15':
                    return 1
                elif ord_date.strftime('%m') == '02' and ord_date.strftime('%d') == '14':
                    return 1
                elif ord_date.strftime('%m') == '02' and ord_date.strftime('%d') =='29':
                    return 1
                elif ord_date.strftime('%m') == '03' and ord_date.strftime('%d') == '17':
                    return 1
                elif ord_date.strftime('%m') == '03' and ord_date.strftime('%d') == '25':
                    return 1
                elif ord_date.strftime('%m') == '03' and ord_date.strftime('%d') == '28':
                    return 1
                elif ord_date.strftime('%m') == '05' and ord_date.strftime('%d') == '08':
                    return 1
                elif ord_date.strftime('%m') == '05' and ord_date.strftime('%d') == '23':
                    return 1
                elif ord_date.strftime('%m') == '06' and ord_date.strftime('%d') == '19':
                    return 1
                elif ord_date.strftime('%m') == '06' and ord_date.strftime('%d') == '21':
                    return 1
                elif ord_date.strftime('%m') == '06' and ord_date.strftime('%d') == '24':
                    return 1
                else:
                    return 0

In [ ]:
holi_fe=preprocessing.FunctionTransformer(holiday_fe)

In [ ]:
def l_m_fe(data):
    pd.DataFrame(data['gmv']).describe(percentiles=[.70,.80,.90]).T
    data['product_type'] = data['gmv'].apply(lambda x:'luxury' if x >= data['gmv'].quantile(.8) else 'mass_market')
    return data

In [ ]:
lux_mass_market=preprocessing.FunctionTransformer(l_m_fe)

In [ ]:
def discount_percent(data):
    col_loc = data.columns.get_loc('list_price') + 1
    data.insert(loc=col_loc, column='Discount%', value = \
    round(100*((data['product_mrp'] - data['list_price']) / data['product_mrp']),2))
    return data


In [ ]:
disc_percent=preprocessing.FunctionTransformer(discount_percent)

### Building Pipeline

In [256]:
feature_eng_pipe=Pipeline([
    # --------- Feature Engineering -------------
    #------addition of required columns------
    ("list_price_fe",list_price_fe),
    ("pay_day_flag_fe",pd_flag_fe),
    ("holiday_fe",holi_fe),
    ("luxury_mass_market",lux_mass_market),      
    ("discount_percentage",disc_percent)    
])


In [257]:
feature_eng_pipe.fit(data)

Pipeline(steps=[('discount_percentage',
                 FunctionTransformer(func=<function discount_percent at 0x000002C8FECBC3A0>))])

In [258]:
feature_eng_pipe.transform(data)

,order_date,Year,Month,gmv,list_price,Discount%,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,payday_flag,occassion_flag,product_type
0,2015-10-17 15:11:54,2015,10,6400.0,6400.0,10.99,1,\N,\N,COD,5,CameraAccessory,CameraAccessory,CameraTripod,7190,0,0,mass_market
1,2015-10-19 10:07:22,2015,10,6900.0,6900.0,4.03,1,\N,\N,COD,7,CameraAccessory,CameraAccessory,CameraTripod,7190,0,0,mass_market
2,2015-10-20 15:45:56,2015,10,1990.0,1990.0,5.19,1,\N,\N,COD,10,CameraAccessory,CameraAccessory,CameraTripod,2099,0,0,mass_market
3,2015-10-14 12:05:15,2015,10,1690.0,1690.0,19.49,1,\N,\N,Prepaid,4,CameraAccessory,CameraAccessory,CameraTripod,2099,1,0,mass_market
4,2015-10-17 21:25:03,2015,10,1618.0,1618.0,22.92,1,\N,\N,Prepaid,6,CameraAccessory,CameraAccessory,CameraTripod,2099,0,0,mass_market
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645038,2015-09-24 10:06:09,2015,9,2990.0,2990.0,64.82,1,\N,\N,COD,11,CameraAccessory,CameraAccessory,Telescope,8500,0,0,mass_market
1645039,2015-09-02 19:32:02,2015,9,2990.0,2990.0,64.82,1,\N,\N,COD,8,CameraAccessory,CameraAccessory,Telescope,8500,0,0,mass_market
1645040,2015-09-14 16:27:04,2015,9,2990.0,2990.0,64.82,1,\N,\N,COD,6,CameraAccessory,CameraAccessory,Telescope,8500,1,0,mass_market
1645041,2015-09-24 12:44:24,2015,9,2990.0,2990.0,64.82,1,\N,\N,COD,6,CameraAccessory,CameraAccessory,Telescope,8500,0,0,mass_market
